In [ ]:
from sklearn.cluster import KMeans
from  sklearn.metrics import silhouette_score
from Utils import *
import cv2
import numpy as np
import matplotlib.pyplot as plt

Segment face

In [ ]:
face_image_path = "../../data/images/greenspillexample.jpg"
class_indices = [2, 3]
face_segmented = segment_face_et_al(og_img_path=face_image_path, class_indices=class_indices)

Convert to HLS and get Hue, Luminosity and Saturation

In [ ]:
hls_face = cv2.cvtColor(face_segmented, cv2.COLOR_BGR2HLS)
hue, lum, sat = cv2.split(hls_face)


Define HLS ranges to produce a skin mask

In [ ]:
lower = np.array([0, 40, 20], dtype=np.uint8)
upper = np.array([80, 220, 255], dtype=np.uint8)
skin_mask = cv2.inRange(hls_face, lower, upper)

skin_pixels = hls_face[skin_mask > 0]


Get KMeans Silhouette score

In [ ]:
sil_scores = []

k_range = range(2, 6)

for k in k_range:
    kmeans = KMeans(n_clusters=k, n_init=10)
    labels = kmeans.fit_predict(skin_pixels)
    score = silhouette_score(skin_pixels, labels)
    sil_scores.append(score)

# Get ideal k clusters
best_index = np.argmax(sil_scores)
best_k = list(k_range)[best_index]
print(f"Best k: {best_k}")


In [ ]:
# Plot silhouette scores
plt.plot(k_range, sil_scores, 'go-')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette score')
plt.title('Optimal cluster count via Silhouette')
plt.grid(True)
plt.show()

Get skin tones

In [ ]:
# Run KMeans
kmeans = KMeans(n_clusters=best_k, n_init=10)
kmeans.fit(skin_pixels)

# Get cluster centres
hue_centres = kmeans.cluster_centers_[:, 0]     # Just hue channel
labels = kmeans.labels_
label_counts = np.bincount(labels)

# Get dominant cluster (based on most pixels)
dominant_idx = labels[np.argmax(label_counts)]
dominant_hue = hue_centres[dominant_idx]

print(f"Dominant skin hue: {dominant_hue :.2f}")    # 16.23

Bin values

In [ ]:
bins = np.linspace(0, 180, 19)  # 18 bins of 10 degrees
bin_labels = np.digitize(hue_centres, bins) - 1

# Initialise bin weights (same length as bins minus 1)
bin_weights = np.zeros(len(bins) - 1)

# Add cluster weights to their corresponding bin
for i, bin_index in enumerate(bin_labels):
    if 0 <= bin_index < len(bin_weights):
        bin_weights[bin_index] += label_counts[i]



Plot histogram

In [ ]:
plt.bar(bins[:-1], bin_weights, width=10, color='coral', edgecolor='black', align='edge')
plt.title("Hue Distribution by KMeans Clusters")
plt.xlabel("Hue (HLS)")
plt.ylabel("Weighted Pixel Count")
plt.grid(True)
plt.show()